In [ ]:
!pip install python-binance

In [ ]:
import sqlalchemy
import asyncio
import datetime
import nest_asyncio
import pandas as pd
from binance import Client
from binance import AsyncClient, BinanceSocketManager

    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:48:59.988  29729.78
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:48:59.989  29729.78


In [ ]:
nest_asyncio.apply()

In [ ]:
api_key = "ncaGKCQNu9KS8kEwF3Jri37YjsOkKCfLGiKZcq87iThEU3QZIWdxOyeQLUvOCPpG"
api_secret = "tvSQ36Idx7W1QwiMx5ycduhziLQZaHUbhnHZGhqlSRaF6VrkP8QqsLY3LCidthdJ"

client = Client(api_key, api_secret)

    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:49:00.004  29729.78


In [ ]:
async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    # start any sockets here, i.e a trade socket
    ts = bm.trade_socket('BTCUSDT')
    # then start receiving messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            frame = createframe(res)
            frame.to_sql('BTCUSDT', engine, if_exists='append', index=False)
            print(frame)

    await client.close_connection()

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:10.929  29729.45
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:11.794  29729.44
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:12.230  29729.45
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:12.382  29729.45
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:12.651  29729.45
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:12.703  29729.45
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:12.828  29729.45
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:12.855  29729.45
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:12.911  29729.45
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:12.962  29729.45
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:13.333  29729.45
    symbol

KeyboardInterrupt: ignored

In [ ]:
def createframe(msg):
  df = pd.DataFrame([msg])
  df = df.loc[:, ['s', 'E', 'p']]
  df.columns = ['symbol', 'Time', 'Price']
  df.Price = df.Price.astype(float)
  df.Time = pd.to_datetime(df.Time, unit='ms')
  time_change = datetime.timedelta(hours=9)
  df.Time = df.Time + time_change 
  return df

In [ ]:
engine = sqlalchemy.create_engine('sqlite:////content/drive/MyDrive/Colab Notebooks/DB/BTCUSDTstream.db')

In [ ]:
df = pd.read_sql('BTCUSDT', engine)
df

    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:23.668  29729.44
    symbol                    Time     Price
0  BTCUSDT 2022-06-05 17:39:23.681  29729.44


,symbol,Time,Price
0,BTCUSDT,2022-06-05 17:35:12.001,29705.97
1,BTCUSDT,2022-06-05 17:35:12.256,29705.98
2,BTCUSDT,2022-06-05 17:35:12.319,29705.98
3,BTCUSDT,2022-06-05 17:35:12.602,29705.98
4,BTCUSDT,2022-06-05 17:35:12.901,29705.98
...,...,...,...
1195,BTCUSDT,2022-06-05 17:39:21.539,29729.45
1196,BTCUSDT,2022-06-05 17:39:21.820,29729.45
1197,BTCUSDT,2022-06-05 17:39:23.244,29729.44
1198,BTCUSDT,2022-06-05 17:39:23.668,29729.44


In [ ]:
#TrendFollowing
#if the crypto was rising by x % -> Buy
#exit when profit is above 0.15% or loss is crossing -0.15%

def strategy(entry, lookback, qty, open_position=False):
  while True:
    df = pd.read_sql('BTCUSDT', engine)
    lookbackperiod = df.iloc[-lookback:]
    cumret = (lookbackperiod.Price.pct_change + 1).cumprod() - 1
    if not open_position:
      if cumret[cumret.last_valid_index()] > entry:
        order = client.create_order(symbol='BTCUSDT', side="BUY", type="MARKET". quantity=qty)
        print(order)
        open_position=True
        break

  if open_position:
    while True:
      df = pd.read_sql('BTCUSDT', engine)
      sincebuy = df.loc[df.Time > pd.to_datetime(order['transactTime'], unit='ms')]

      if len(sincebuy) > 1:
        sincebuyret = (sincebuy.Price.pct_change() + 1).cumprod() - 1
        last_entry = sincebuyret[sincebuyret.last_valid_index()]
        if last_entry > 0.0015 or last_entry < -0.0015:
          order = client.create_order(symbol='BTCUSDT', side="SELL", type="MARKET". quantity=qty)
          print(order)
          break